In [1]:
import os
import pandas as pd
import os.path as osp
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Load files

In [2]:
os.getcwd()

'D:\\Project- Electro\\social_behavior_electro\\data_organization\\python'

In [3]:
folder_path = osp.join('..','..','analysis')
path_to_aversive_enc_pre = osp.join(folder_path,
                            'Population analysis results for LFP Coherence - Encounter-PreEncounter_4-12_30-80_aversive.xlsx')
path_to_affiliative_enc_pre = osp.join(folder_path,
                               'Population analysis results for LFP Coherence - Encounter-PreEncounter_4-12_30-80_affiliative.xlsx')

path_to_aversive_post_pre = osp.join(folder_path,
                            'Population analysis results for LFP Coherence - PostEncounter-PreEncounter_4-12_30-80_aversive.xlsx')
path_to_affiliative_post_pre = osp.join(folder_path,
                               'Population analysis results for LFP Coherence - PostEncounter-PreEncounter_4-12_30-80_affiliative.xlsx')



path_to_affiliative_lfp = osp.join(folder_path,
                            'lfp_rawdata_affiliative.xlsx') 

path_to_aversive_lfp = osp.join(folder_path,
                            'lfp_rawdata_aversive.xlsx')

path_to_affiliative_spikes = osp.join(folder_path,
                            'spikes_rawdata_affiliative.xlsx') 

path_to_aversive_spikes = osp.join(folder_path,
                            'spikes_rawdata_aversive.xlsx')


path_to_common_areas = osp.join(folder_path, 'common_area_pairs_w5_sessions.xlsx')



In [5]:
MIN_N_SESSIONS = 5
METRIC = 'kappa'

In [6]:
df_aff_lfp_raw = pd.read_excel(path_to_affiliative_lfp ,header=[0,1,2], index_col=[0])
df_avv_lfp_raw = pd.read_excel(path_to_aversive_lfp ,header=[0,1,2], index_col=[0])

In [7]:
df_aff_spikes_raw = pd.read_excel(path_to_affiliative_spikes ,header=[0,1], index_col=[0])
df_avv_spikes_raw = pd.read_excel(path_to_aversive_spikes ,header=[0,1], index_col=[0])

In [8]:
# df_aff_spikes_raw.columns

In [9]:
df_aff_enc_pre = pd.read_excel(path_to_affiliative_enc_pre,None)
df_avv_enc_pre = pd.read_excel(path_to_aversive_enc_pre,None)

df_aff_post_pre = pd.read_excel(path_to_affiliative_post_pre,None)
df_avv_post_pre = pd.read_excel(path_to_aversive_post_pre,None)


df_files_avv_bad = pd.read_excel(path_to_aversive_enc_pre,'Uploaded files', header=None)
df_files_aff_bad = pd.read_excel(path_to_affiliative_enc_pre,'Uploaded files', header=None)

df_areas = pd.read_excel(path_to_common_areas)


## Clean and combine data

In [10]:
import re

def fix_file_paths(df_files):
    
    df_files = df_files.rename(columns={df_files.columns[1]:'timestamps', df_files.columns[0]:'lfp', })
    
    return df_files
df_files_avv = fix_file_paths(df_files_avv_bad)['lfp']
df_files_aff = fix_file_paths(df_files_aff_bad)['lfp']


def extract_ratnum_from_file_name(filename):
    filename = filename.replace(' ','')
    match = re.search(r"rat(\d{1,2})", filename.lower())

    if match:
        number = int(match.group(1))       
    else:
        number = -1
    return number

def extract_daynum_from_file_name(filename):
    filename = filename.replace(' ','')
    match = re.search(r"day(\d{1,2})", filename.lower())
    if match:
        number = int(match.group(1))       
    else:
        number = -1
    return number
# rat_numbers = [extract_ratnum_from_file_name(filename) for filename in filenames]


In [11]:
area_num_to_name_map = {
    '111': 'MeD',
    '2': 'MePV',
    '13':'CeA',
    '112': 'BMA',
    '14': 'AA',
    '16': 'EA',
    '12': 'STIA',
    '15': 'VP'
}



def reformat_columns(df):
    # combine column names and remove levels
    df_new = pd.DataFrame()
    areas = df.columns.get_level_values('area').unique()
    freqs = df.columns.get_level_values('freq').unique()
    stages = df.columns.get_level_values('stage').unique()

    df_new.index = df.index
    for area in areas:
        for freq in freqs:
            for stage in stages:

                col_name = (area, freq, stage)
                
                area_name = area_num_to_name_map[area]
                if "diffduring" in stage.lower():
                    new_stage = 'enc_pre_lfp'
                elif"diffafter" in stage.lower():
                    new_stage = 'post_pre_lfp'
                else:
                    continue
                
                new_freq = freq.replace('-','_') + 'Hz'
                
                
                new_col_name = '_'.join((area_name, new_freq, new_stage))

                df_new[new_col_name] = df[col_name]
    return df_new


SPIKE_DATA_TYPE_SRC = 'diff'
SPIKE_DATA_TYPE_TARGET = 'spike_diff'

# SPIKE_DATA_TYPE_SRC = 'ratio'
# SPIKE_DATA_TYPE_TARGET = 'spike_ratio'

# SPIKE_DATA_TYPE_SRC = 'spikes_'
# SPIKE_DATA_TYPE_TARGET = 'spike_raw'

def reformat_columns_spikes(df):
    # combine column names and remove levels
    df_new = pd.DataFrame()
    areas = df.columns.get_level_values('area').unique()
    stages = df.columns.get_level_values('stage').unique()

    df_new.index = df.index
    add_pre_sufix = 'ratio' in SPIKE_DATA_TYPE_TARGET or 'diff' in SPIKE_DATA_TYPE_TARGET
    suffix = 'pre_' if add_pre_sufix else ''
    for area in areas:
        for stage in stages:

            col_name = (area, stage)

            area_name = area_num_to_name_map[area]
            if SPIKE_DATA_TYPE_SRC+"during" in stage.lower():
                new_stage = 'enc_'+suffix+ SPIKE_DATA_TYPE_TARGET
            elif SPIKE_DATA_TYPE_SRC+"after" in stage.lower():
                new_stage = 'post_'+suffix+ SPIKE_DATA_TYPE_TARGET
            elif SPIKE_DATA_TYPE_SRC+"before" in stage.lower():
                new_stage = 'pre_'+ SPIKE_DATA_TYPE_TARGET
            else:
                continue

            

            new_col_name = '_'.join((area_name, new_stage))

            df_new[new_col_name] = df[col_name]
    return df_new

def reformat_columns_spikes_v2(df):
    # combine column names and remove levels
    df_new = pd.DataFrame()
    areas = df.columns.get_level_values('area').unique()
    stages = df.columns.get_level_values('stage').unique()

    df_new.index = df.index
    add_pre_sufix = 'ratio' in SPIKE_DATA_TYPE_TARGET or 'diff' in SPIKE_DATA_TYPE_TARGET
    suffix = 'enc' if add_pre_sufix else ''
    for area in areas:
        for stage in stages:

            col_name = (area, stage)

            area_name = area_num_to_name_map[area]
            if SPIKE_DATA_TYPE_SRC+"during" in stage.lower():
                during_data = df[col_name].dropna().values
            elif SPIKE_DATA_TYPE_SRC+"after" in stage.lower():
                after_data = df[col_name].dropna().values
            elif SPIKE_DATA_TYPE_SRC+"before" in stage.lower():
                before_data = df[col_name].dropna().values
            else:
                continue

            
        # create new column with calculation
        log_during_change = np.log(during_data) - np.log(before_data)
        
        
        new_stage = 'enc_pre_log_'+ SPIKE_DATA_TYPE_TARGET
        new_col_name = '_'.join((area_name, new_stage))
        df_new[new_col_name] = df[col_name]
        
        log_after_change = np.log(after_data) - np.log(before_data)
        new_stage = 'post_pre_log'+ SPIKE_DATA_TYPE_TARGET
        df_new[new_col_name] = df[col_name]
    return df_new

In [12]:
# df_avv_spikes

In [13]:
df_aff_lfp = reformat_columns(df_aff_lfp_raw)
df_avv_lfp = reformat_columns(df_avv_lfp_raw)
df_aff_spikes = reformat_columns_spikes(df_aff_spikes_raw)
df_avv_spikes = reformat_columns_spikes(df_avv_spikes_raw)
# df_aff_spikes = reformat_columns_spikes_v2(df_aff_spikes_raw)
# df_avv_spikes = reformat_columns_spikes_v2(df_avv_spikes_raw)

# Add variables befor concatination
df_aff_enc_pre = pd.read_excel(path_to_affiliative_enc_pre,None)
# df_aff_enc_pre['sociability'] = 'affiliative'
# df_aff_enc_pre['stage'] = 'enc_pre'

df_aff_post_pre = pd.read_excel(path_to_affiliative_post_pre,None)
# df_aff_post_pre['sociability'] = 'affiliative'
# df_aff_post_pre['stage'] = 'post_pre'


df_avv_enc_pre = pd.read_excel(path_to_aversive_enc_pre,None)
# df_avv_enc_pre['sociability'] = 'aversive'
# df_avv_enc_pre['stage'] = 'enc_pre'


df_avv_post_pre = pd.read_excel(path_to_aversive_post_pre,None)
# df_avv_post_pre['sociability'] = 'aversive'
# df_avv_post_pre['stage'] = 'post_pre'



In [14]:
freq_keywords = {'First': '4_18Hz', 'Second':'30_80Hz'}
stage_keywords = {'During':'enc_pre', 'After':'post_pre'}
substrings_to_remove = ['During', 'After','Before', 'First', 'Second']
ignore_keyworks = ['Norm', 'files']

def remove_columns_with_fewer_values(df, N=5):
    # Get the count of non-null values in each column
    column_counts = df.count()

    # Filter columns based on count condition
    columns_to_remove = column_counts[column_counts < N].index

    # Drop the columns from the DataFrame
    updated_df = df.drop(columns=columns_to_remove)

    return updated_df

def reformat_dict_to_table(df_dict, file_df):
# def reformat_dict_to_table(df_dict, freq_keywords, stage_keywords, ignore_keyworks, substrings_to_remove):
    ret_df_list = []
    for sheet, df in df_dict.items():
        df = df.copy()
        # ignore first and last sheets that contain a summary and list of file names
        if any(substring in sheet for substring in ignore_keyworks): 
            continue
            
        if isinstance(df, str):
            print(df)
            continue
            
        if len(df) == 0:
            continue
        
         # Remove all substrings to keep just the area name
        area_name = sheet
        for sub_string_to_remove in substrings_to_remove:
            area_name = area_name.replace(sub_string_to_remove,'')
        
        for fk, freq in  freq_keywords.items():
            if fk in sheet:
                this_freq = freq_keywords[fk]
            for sk in stage_keywords.keys():
                if sk in sheet:
                    this_stage = stage_keywords[sk]
        
                # Rename all column names to match
        
        for col in df.columns:
            area_pair = [col, area_name]
            area_pair.sort()
            
            df.rename(columns={col:f'{area_pair[0]}_{area_pair[1]}_{this_freq}_{this_stage}'}, inplace=True)
        
        
        df.dropna(axis=1, how='all', inplace=True)
        df = remove_columns_with_fewer_values(df)
        ret_df_list.append(df)
        
    df_ret = pd.concat(ret_df_list,axis=1)
    df_ret['files'] = file_df.values
    return df_ret
                
            



In [15]:
df_aff_enc_pre_rectified = reformat_dict_to_table(df_aff_enc_pre, df_files_aff)
df_aff_enc_post_rectified = reformat_dict_to_table(df_aff_post_pre, df_files_aff)
df_aff = pd.concat([
    df_aff_enc_pre_rectified.set_index('files'),
    df_aff_enc_post_rectified.set_index('files'),
    df_aff_lfp,
    df_aff_spikes
],
    axis=1)


df_aff['sociability'] = 'affiliative'


df_avv_enc_pre_rectified = reformat_dict_to_table(df_avv_enc_pre, df_files_avv)
df_avv_enc_post_rectified = reformat_dict_to_table(df_avv_post_pre, df_files_avv)
df_avv = pd.concat([
    df_avv_enc_pre_rectified.set_index('files'),
    df_avv_enc_post_rectified.set_index('files'),
    df_avv_lfp,
    df_avv_spikes
], axis=1)

df_avv['sociability'] = 'aversive'


df_avv_cols = df_avv.columns
df_aff_cols = df_aff.columns
common_cols = set(df_aff_cols).intersection(set(df_avv_cols))
df_aff = df_aff[common_cols]
df_avv = df_avv[common_cols]


C:\Users\user\AppData\Local\Temp\ipykernel_7072\3964125948.py:30: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_aff = df_aff[common_cols]
C:\Users\user\AppData\Local\Temp\ipykernel_7072\3964125948.py:31: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_avv = df_avv[common_cols]


In [16]:
# Fix column names
# change 4-12 to 4-18
def fix_range_name(s):
    return s.replace('4_18','4_12')
# if lfp is in the name, put it at the beggining
# if spike_ratio is in the name, put it at the beginning
def move_suffix_to_apx(s, sfx):
    if sfx in s:
        s = s.replace('_'+sfx, '')
        s = sfx+'_'+s
    return s

# if there is not lfp or skipe ration in the name, add coherence at the befinning
def add_cohherence(s):
    if 'lfp' not in s and 'spike' not in s and ('enc' in s or 'post' in s):
        s = 'coherence_' + s
    return s

# add number to catergorical variables to ensure that they are first
def add_number(s):
    if 'sociability' in s:
        s = '0_' + s
        
    elif 'rat_number' in s:
        s = '1_' + s
        
    elif 'day_number' in s:
        s = '2_' + s
    return s

def add_all_fix_functions(s):
    s = fix_range_name(s)
    s = move_suffix_to_apx(s, 'lfp')
    s = move_suffix_to_apx(s, SPIKE_DATA_TYPE_TARGET)
    s = add_cohherence(s)
#     s = add_number(s)
    return s

all_df = pd.concat([df_aff, df_avv])
filenames = all_df.index.to_list()
all_df['rat_number'] = pd.Categorical([extract_ratnum_from_file_name(r) for r in filenames])
all_df['day_number'] = pd.Categorical([extract_daynum_from_file_name(r) for r in filenames])


all_df.rename(columns={c:add_all_fix_functions(c) for c in all_df.columns} , inplace=True)
all_df = all_df.reindex(sorted(all_df.columns), axis=1)

In [17]:
cat_vars  = ['sociability','rat_number','day_number']

df_spike_data = all_df[[c for c in all_df.columns if 'spike' in c or c in cat_vars]]
df_lfp_data = all_df[[c for c in all_df.columns if 'lfp' in c or c in cat_vars]]
df_coherence_data = all_df[[c for c in all_df.columns if 'coherence' in c or c in cat_vars]]
df_coherence_lfp = all_df[[c for c in all_df.columns if 'spike' not in c]]

## Preliminary feature selection via statistical analysis

In [50]:
from scipy.stats import mannwhitneyu, ttest_ind

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, StratifiedGroupKFold

from sklearn.preprocessing import StandardScaler


from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


# ############################
# import sklearn.neighbors._base
# import sys
# sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# from missingpy import MissForest
# ############################

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier


from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from scipy.special import logit


import traceback
from tqdm import tqdm
import warnings


import itertools
import matplotlib.pyplot as plt

import sweetviz as sv


In [19]:
def calculate_mean_variance(dataframe):
    column_names = dataframe.columns
    result = pd.DataFrame(columns=['Column', 'Mean', 'Variance'])

    for column in column_names:
        mean_value = dataframe[column].mean()
        variance_value = dataframe[column].var()
        result = result.append({'Column': column, 'Mean': mean_value, 'Variance': variance_value}, ignore_index=True)

    return result

# result_df = calculate_mean_variance(df_spike_data[[c for c in df_spike_data if c not in cat_vars]])
# print(result_df)

In [20]:
def plot_correlation_heat_map(df):
    corr_data = df.corr()
    a4_dims = (11.7, 8.27)
    fig, ax = plt.subplots(figsize=a4_dims)
    g = sns.heatmap(corr_data, ax=ax)
    return

def mark_nans_with_x(correlation_matrix, ax):
#     correlation_matrix = correlation_matrix.drop(cat_vars, axis = 1)
    # Step 3: Extract variables with all NaN values
    nan_variables = correlation_matrix.columns[correlation_matrix.isnull().all()]

    # Step 4: Mark relevant squares with NaN values in the heat map
    for i in range(len(correlation_matrix.columns)):
        for j in range(len(correlation_matrix.columns)):
            if correlation_matrix.iloc[i, j] != 1.0 and pd.isnull(correlation_matrix.iloc[i, j]):
                ax.text(j + 0.5, i + 0.5, 'X', ha='center', va='center', color='red', fontsize=16)
    return
    

def plot_correlation_heat_map_sidebyside(df):
    # Convert numeric columns to float if needed
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    df[numeric_columns] = df[numeric_columns].astype(float)

    # Step 1: Split the dataframe based on the value of "sociability"
    df_low_sociability = df[df["sociability"] == "affiliative"]
    df_high_sociability = df[df["sociability"] == "aversive"]

    # Step 2: Extract correlation matrices
    corr_low_sociability = df_low_sociability.corr()
    corr_high_sociability = df_high_sociability.corr()

    # Step 3: Create heat map plot side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    sns.heatmap(corr_low_sociability, annot=False, cmap='coolwarm', vmin=-1, vmax=1, ax=ax1)
    sns.heatmap(corr_high_sociability, annot=False, cmap='coolwarm', vmin=-1, vmax=1, ax=ax2)

    # Step 4: Mark relevant squares with NaN values in the heat maps
    for ax, corr_matrix in zip([ax1, ax2], [corr_low_sociability, corr_high_sociability]):
        nan_matrix = corr_matrix.isnull().astype(int)
        for i in range(len(corr_matrix.index)):
            for j in range(len(corr_matrix.columns)):
                if nan_matrix.iloc[i, j] == 1:
                    ax.text(j + 0.5, i + 0.5, 'X', ha='center', va='center', color='red', fontsize=16)

    # Step 5: Extract and mark cells with significant difference
    for i in range(len(corr_low_sociability.index)):
        for j in range(len(corr_low_sociability.columns)):
            if i != j:
                
                low_col = pd.to_numeric(df_low_sociability.iloc[:, i], errors='coerce')
                high_col = pd.to_numeric(df_high_sociability.iloc[:, j], errors='coerce')
                
                len_high= np.sum(~np.isnan(high_col.values))
                len_low= np.sum(~np.isnan(low_col.values))
                
                if len_high<MIN_N_SESSIONS or len_low<MIN_N_SESSIONS:
                    p_value = 1
#                     print(f'len_high: {len_high}, len_high: {len_low}')
                else:
#                     _, p_value = mannwhitneyu(low_col, high_col, alternative='two-sided')
                    p_value = ttest_ind(low_col, high_col, nan_policy='omit').pvalue
                if p_value < 0.05:
                    ax1.text(j + 0.5, i + 0.5, 'O', ha='center', va='center', color='blue', fontsize=16)
                    ax2.text(j + 0.5, i + 0.5, 'O', ha='center', va='center', color='blue', fontsize=16)
#                 print(f'i,j:({i},{j})')
    
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.7, hspace=None)
    


  



In [21]:
# plot_correlation_heat_map_sidebyside(df_spike_data)

In [22]:
# #EDA using Autoviz
# df_vis = df_coherence_data
# df_vis = df_vis[[c for c in df_vis.columns if c not in cat_vars]]

# df_vis = df_vis.apply(pd.to_numeric)
# df_vis['sociability'] = df_coherence_data['sociability']

# # feature_config = sv.FeatureConfig(force_num=[c for c in df_vis.columns if c not in cat_vars])
# sweet_report = sv.compare_intra(
#     df_vis,df_vis['sociability']=='affiliative',
#     ['affiliative', 'aversive'],
# #     feat_cfg=feature_config
# )

# #Saving results to HTML file
# sweet_report.show_html('df_coherence_data.html')
# # sweet_report.show_notebook(filepath='sweet_report.html')

In [23]:
%%capture
def transform_var(func_handle, X_train,X_test, method='reuse', func_parameters=None):
    if func_parameters is None:
        func_train = func_handle()
    else:
        func_train = func_handle(**func_parameters)
            
    X_train_trans = func_train.fit_transform(X_train)
    
    if method == 'reuse':
        X_test_trans = func_train.transform(X_test)
        func_test = func_train
    # create new instance and refit on test data
    elif method == 'new':
        if func_parameters is None:
            func_test = func_handle()
        else:
            func_test = func_handle(**func_parameters)
        X_test_trans = func_test.fit_transform(X_test)
    else:
        raise ValueError("Invalid input value. The value must be either 'reuse' or 'new'.")
    return X_train_trans, X_test_trans, func_train, func_test

In [134]:
from sklearn.metrics import cohen_kappa_score

def eval_model(y_test, y_test_pred, y_prob=None):
#     print(f'y:{y_test}')
#     print(f'y_pred:{y_test_pred}')
    # Calculating evaluation metrics on the testing set
    ret_accuracy = accuracy_score(y_test, y_test_pred)
    ret_precision = precision_score(y_test, y_test_pred, pos_label="affiliative")
    ret_recall = recall_score(y_test, y_test_pred,pos_label="affiliative")
    ret_f1 = f1_score(y_test, y_test_pred,pos_label="affiliative")
    
    kappa = cohen_kappa_score(y_test, y_test_pred)
#     print(y_test)
#     print(y_test=='affiliative')
    if y_prob is not None:
        roc_auc = roc_auc_score(y_test=='affiliative', y_prob)
    ret = {
        'accuracy':np.round(ret_accuracy, 3),
        'precision':np.round(ret_precision, 3),
        'recall':np.round(ret_recall, 3),
        'f1':np.round(ret_f1, 3),
        'kappa': np.round(kappa, 3),
        'roc_auc':roc_auc,
    }
    

    return ret

def sum_model_results(y, y_pred, 
                      confidence_levels, affiliative,
                      filenames,
                      dataset):
    df_results = pd.DataFrame()
    df_results['GT'] = y
    df_results['predicted'] = y_pred
    df_results['correct'] = df_results['GT'] == df_results['predicted']
    df_results['confidence'] = confidence_levels
    df_results['affiliative_level'] = affiliative
    df_results['filenames'] = filenames

    return df_results

def sum_all_results(y_train, y_train_pred, y_test, y_test_pred, **kwargs):
    
    ret_train = eval_model(y_train,y_train_pred)
    ret_test = eval_model(y_test,y_test_pred)
    df_res_all = pd.DataFrame.from_dict([ret_test, ret_train])
    df_res_all.index = ['test','train']
    
    for param, vals in kwargs.items():
        try:
            df_res_all[param] = [vals]*2
        except:
            traceback.print_exc()
    return df_res_all


def train_eval_model(config_dict,
                     X_train, y_train, X_test, y_test,
                     train_file_names, test_file_names,
                     method_name, imputer_name, scaler_name, classifier_name,
                     k_fold, ind, iteration):
    
#     print('I am here')
    scaler = config_dict['scaler'][scaler_name]
    imputer_class = config_dict['imputer'][imputer_name]
    
    if ('imputer_param' in config_dict) and (imputer_name in config_dict['imputer_param']):
        imputer_param = config_dict['imputer_param'][imputer_name]
    else:
        imputer_param = None
    
    classifier_class = config_dict['model'][classifier_name]

    
    # impute
    
    X_train_imputed, X_test_imputed, imputer_train, imputer_test = transform_var(
        imputer_class, X_train, X_test, method_name, imputer_param);

    # ==================== impute each class seperatly =================
#     X_train_aff = X_train[y_train=='affiliative']
#     X_test_aff = X_test[y_test=='affiliative']
#     X_train_avv = X_train[y_train=='aversive']
#     X_test_avv = X_test[y_test=='aversive']
    
#     print(f'size of X_train_aff :{X_train_aff.shape}')
#     print(f'size of X_test_aff :{X_test_aff.shape}')
#     print(f'size of X_train_avv :{X_train_avv.shape}')
#     print(f'size of X_test_avv :{X_test_avv.shape}')
    
    
    
#     X_train_aff_imputed, X_test_aff_imputed, imputer_train, imputer_test = transform_var(
#         imputer_class, X_train_aff, X_test_aff, method_name, imputer_param);
    
#     X_train_avv_imputed, X_test_avv_imputed, imputer_train, imputer_test = transform_var(
#         imputer_class, X_train_avv, X_test_avv, method_name, imputer_param);
    
#     X_train_imputed = np.concatenate((X_train_aff_imputed,X_train_avv_imputed), axis=0)
#     X_test_imputed = np.concatenate((X_test_aff_imputed,X_test_avv_imputed), axis=0)
    # ==================== impute each class seperatly (end) =================


    # scale
    X_train_scaled, X_test_scaled, scaler_train, scaler_test= transform_var(
        imputer_class, X_train_imputed, X_test_imputed, method_name)

    # train classifier
    if ('model_param' in config_dict) and (classifier_name in config_dict['model_param']):
        model = classifier_class(**config_dict['model_param'][classifier_name])
    else: 
        model = classifier_class()
    
    model.fit(X_train_scaled, y_train)

    # predict

    y_train_pred = model.predict(X_train_scaled)

    y_test_pred = model.predict(X_test_scaled)

    y_test_pred_prob = model.predict_proba(X_test_scaled)
    y_train_pred_prob = model.predict_proba(X_train_scaled)

    test_confidence_levels = y_test_pred_prob.max(axis=1)
    train_confidence_levels = y_train_pred_prob.max(axis=1)

    test_affiliative = y_test_pred_prob[:,0]
    train_affiliative = y_train_pred_prob[:,0]

    sum_results = sum_all_results(
        y_train, y_train_pred, y_test, y_test_pred,
        method = method_name,
        imputer= imputer_name,
        scaler = scaler_name, 
        model = classifier_name,
        k_fold = k_fold,
        ind = ind,
        trained_model = model,
        trained_imputer = imputer_train,
        trained_scalar = scaler_train,
        iteration = iteration
        
    )

    df_results_train = sum_model_results(
        y_train, y_train_pred,train_confidence_levels, train_affiliative,train_file_names ,dataset='train')
    df_results_train['dataset'] = 'train'
    df_results_test = sum_model_results(y_test, y_test_pred,test_confidence_levels, test_affiliative, test_file_names ,dataset='test')
    df_results_test['dataset'] = 'test'
    df_results_all = pd.concat([df_results_test,df_results_train])
    df_results_all
    
    df_results_all['method'] = method_name
    df_results_all['imputer'] = imputer_name
    df_results_all['scaler'] = scaler_name
    df_results_all['model'] = classifier_name
    df_results_all['k_fold'] = k_fold
    df_results_all['ind'] = ind
    df_results_all['iteration'] = iteration
    

    return sum_results, df_results_all, model, imputer_train

def model_selection(config_dict, X, y, subject_id, filenames, n_iterations,verbose=True):
    all_results_summary = []
    all_raw_results = []
    errors_log = []
    models= []
    imputers = []
    for iteration in tqdm(range(n_iterations), disable=n_iterations<10):          
        for k_fold in config_dict['k_fold']:
#             print(f'k_fold: {k_fold}')
#             stratified_group_kfold = StratifiedGroupKFold(n_splits=k_fold, shuffle=True)
#             for ind, (train_index, test_index) in enumerate(stratified_group_kfold.split(X, y, groups=subject_id)):
            stratified_group_kfold = StratifiedKFold(n_splits=k_fold, shuffle=True)        
            for ind, (train_index, test_index) in enumerate(stratified_group_kfold.split(X, y)):
            
            



                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]
                train_file_names, test_file_names = filenames[train_index], filenames[test_index]
                # mamke sure that there is nan columns:
                X_train_nan_flag = np.any(np.all(np.isnan(X_train), axis=0))
                X_test_nan_flag = np.any(np.all(np.isnan(X_test), axis=0))
                if X_train_nan_flag or X_test_nan_flag:
#                     if (X_train_nan_flag):
#                         print(f'Detected nan columns: {np.isnan(X_train)}')
#                     if (X_test_nan_flag):
#                         print(f'Detected nan columns: {np.isnan(X_test)}')
                                                       
                                                       
                    continue

                for method_name in config_dict['method']:
                    for imputer_name in config_dict['imputer'].keys():
                        for scaler_name in config_dict['scaler'].keys():
                            for classifier_name in config_dict['model'].keys():

#                                 try:
                                sum_results, raw_results, model, imputer = train_eval_model(
                                    config_dict,
                                    X_train, y_train, X_test, y_test,
                                    train_file_names, test_file_names,
                                    method_name,
                                    imputer_name,
                                    scaler_name,
                                    classifier_name,
                                    k_fold, ind, iteration)
                                all_results_summary.append(sum_results)
                                all_raw_results.append(raw_results)
                                models.append(model)
                                imputers.append(imputer)
                                

    return all_results_summary, all_raw_results, errors_log,models, imputers
                        
    

In [69]:

def variable_significance_table(df_in):
    significance_table = pd.DataFrame(columns=["Variable", "Statistic", "p-value"])
    df = df_in[[c for c in df_in.columns if c not in cat_vars or 'sociability' in c]]
    
    predictor = df["sociability"]

    for column in df.columns:
        if column != "sociability":
            
            variable = df[column].values
            
            variable_aff = variable[np.where(predictor=='affiliative')]
            variable_avv = variable[np.where(predictor=='aversive')]
            
            
            variable_aff = variable_aff[~pd.isnull(variable_aff)]
            variable_avv = variable_avv[~pd.isnull(variable_avv)]
            
            if len(variable_aff)<MIN_N_SESSIONS or len(variable_avv)< MIN_N_SESSIONS:
                continue
            
            statistic, p_value = mannwhitneyu(variable_avv, variable_aff)
#             p_value = ttest_ind(variable_avv, variable_aff).pvalue
#             statistic = ttest_ind(variable_avv, variable_aff).statistic

            significance_table = significance_table.append(
                {"Variable": column, "Statistic": statistic, "p-value": p_value},
                ignore_index=True
            )
    significance_table = significance_table.sort_values('p-value')
    return significance_table


sig_table_all = variable_significance_table(df_in=all_df)
# sig_table_spike = variable_significance_table(df=df_spike_data[[c for c in df_spike_data.columns if c not in cat_vars or 'sociability' in c]])

# sig_table_all


C:\Users\user\AppData\Local\Temp\ipykernel_7072\640660092.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  significance_table = significance_table.append(
C:\Users\user\AppData\Local\Temp\ipykernel_7072\640660092.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  significance_table = significance_table.append(
C:\Users\user\AppData\Local\Temp\ipykernel_7072\640660092.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  significance_table = significance_table.append(
C:\Users\user\AppData\Local\Temp\ipykernel_7072\640660092.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  significance_table = significance_table.append(
C:\Users

In [70]:
print(sig_table_all.to_string())
# print(sig_table_spike.to_string())

                               Variable  Statistic   p-value
44               lfp_EA_30_80Hz_enc_pre       12.0  0.011655
32               lfp_AA_30_80Hz_enc_pre       41.0  0.017954
17   coherence_CeA_MeD_30_80Hz_post_pre       61.0  0.018207
6       coherence_AA_MeD_4_12Hz_enc_pre      110.0  0.020729
34                lfp_AA_4_12Hz_enc_pre       43.0  0.023270
18     coherence_CeA_MeD_4_12Hz_enc_pre       60.0  0.024456
4      coherence_AA_MeD_30_80Hz_enc_pre      108.0  0.028108
40              lfp_CeA_30_80Hz_enc_pre       20.0  0.031420
72              spike_diff_STIA_enc_pre       66.0  0.044931
19    coherence_CeA_MeD_4_12Hz_post_pre       57.0  0.052898
20   coherence_CeA_STIA_30_80Hz_enc_pre       41.0  0.055278
55             lfp_MePV_4_12Hz_post_pre       74.0  0.066684
42               lfp_CeA_4_12Hz_enc_pre       24.0  0.069096
48              lfp_MeD_30_80Hz_enc_pre      235.0  0.070985
16    coherence_CeA_MeD_30_80Hz_enc_pre       55.0  0.083064
5     coherence_AA_MeD_3

In [71]:
def run_per_param(var_names, all_df,config_dict, n_iterations, verbose=True):

    X = all_df[var_names].values

    samples_to_remove = ~np.all(np.isnan(X),axis=1)
    y = all_df.iloc[samples_to_remove]['sociability'].values
    filenames = all_df.iloc[samples_to_remove].index.values
    subject_id = all_df.iloc[samples_to_remove]['rat_number'].values
    X = all_df.iloc[samples_to_remove][var_names].values
    

    ret, raw_results, errors_log, models, imputers = model_selection(
        config_dict=config_dict,
        X=X, y=y,
        subject_id=subject_id,
        filenames=filenames,
        n_iterations=n_iterations,
        verbose=verbose
    )

#     print(len(models))

    raw_results_df = pd.concat(raw_results)
    
    ret_df = pd.concat(ret)
    ret_df = ret_df.reset_index().rename(columns={'index':'dataset'})
#     mean_df = ret_df.drop(['trained_model','trained_imputer','trained_scalar','ind'],axis=1).groupby(['dataset','method', 'imputer', 'scaler', 'model', 'k_fold']).mean().reset_index()
    mean_df = ret_df.drop(['trained_model','trained_imputer','trained_scalar','ind','iteration'],axis=1).groupby(
        ['dataset','method', 'imputer', 'scaler', 'model', 'k_fold']).mean().reset_index()

    # Add a new column 'ind' that counts the number of rows for each group
    mean_df['ind'] = ret_df.drop(
        ['trained_model','trained_imputer','trained_scalar','ind','iteration'],axis=1).groupby(
        ['dataset','method', 'imputer', 'scaler', 'model', 'k_fold']).size().reset_index(name='ind')['ind']

    return mean_df, raw_results_df, ret_df, errors_log, models, imputers

def select_parameters(config_dict, df,sig_table=None,sig_cut_off=0.1, verbose=False, n_iterations=1, metric='f1'):
   
    if sig_table is None:
        sig_table = variable_significance_table(df)
    
    all_errors_log = []
    best_metric = 0
    best_sum_table = []
    best_raw_rable = []
    best_ret_df = []
    best_model = []
    best_imputer = []
    best_params = dict()
    n_var_start = 1
    sig_table_filt = sig_table.loc[sig_table['p-value']<sig_cut_off]
    
    n_params = len(sig_table_filt)
    all_vars = list(sig_table_filt.Variable.values)
    
    current_vars = all_vars[:n_var_start]
#     current_vars = []
    vars_to_test = all_vars[n_var_start:]
    
    for n_var, new_var in enumerate(vars_to_test):
        var_names = current_vars + [new_var]
        try:
            if verbose:
                print(f"============ New param: {new_var} ({n_var+1} of {len(vars_to_test)}) ============")
            
            mean_df, raw_results_df, ret_df, errors_log, models, imputers = run_per_param(
                var_names,df,config_dict, n_iterations, verbose)
            this_best_sum_df = mean_df.loc[mean_df['dataset']=='test'].sort_values(metric, ascending=False).iloc[0]
            all_errors_log.append(errors_log)
            this_best_metric = mean_df.loc[mean_df['dataset']=='test'][metric].max()
            best_ind = mean_df.loc[mean_df['dataset']=='test'][metric].argmax()
            print(best_metric)
            if this_best_metric > best_metric:
                current_vars.append(new_var)
                if verbose:
                    print(f'*********New best {metric} was found: {this_best_metric}************')
                    print(f'Adding: {new_var}')
                    print(f'Current vars: {current_vars}')
                    print(f'method:{this_best_sum_df.method}')
                    print(f'imputer:{this_best_sum_df.imputer}')
                    print(f'model:{this_best_sum_df.model}')
                    print(f'k_fold:{this_best_sum_df.k_fold}')

                best_metric = this_best_metric
                best_sum_table = mean_df
                best_ret_df = ret_df
                best_raw_rable = raw_results_df.loc[
                    (raw_results_df['method'] == this_best_sum_df.method)&
                    (raw_results_df['imputer'] == this_best_sum_df.imputer)&
                    (raw_results_df['scaler'] == this_best_sum_df.scaler)&
                    (raw_results_df['model'] == this_best_sum_df.model)&
                    (raw_results_df['k_fold'] == this_best_sum_df.k_fold)
                ]
    #                 best_model = models[best_ind]
    #                 best_imputer = imputers[best_ind]
                best_params = {
                    'method': this_best_sum_df.method,
                    'imputer': this_best_sum_df.imputer,
                    'scaler': this_best_sum_df.scaler,
                    'model': this_best_sum_df.model,
                    'k_fold': this_best_sum_df.k_fold
                }
            else:
                print(f'Variable {new_var} did not improve the model')
        except :
            print('=================Error - start ==================')
            traceback.print_exc()
            print('=================Error - end ==================')
            continue
    print('*************=============ALL DONE============*******************')
    return best_metric, best_sum_table,best_raw_rable, current_vars,best_ret_df, all_errors_log, best_model, best_imputer, best_params


In [72]:
# run again

def get_config_dict(config_dict, best_params):
    config_dict_best = config_dict.copy()
    for pname, param in best_params.items():
        if pname == 'method':
            continue
        if pname == 'k_fold':
            config_dict_best[pname] = [param]
        else:
            config_dict_best[pname] = {param:config_dict[pname][param]}
    return config_dict_best
    
def run_model_again(
    df, list_of_variables,
    best_params,
    config_dict, 
    verbose=False,
    n_total_iterations=100,
    run_shuffled=False,
    n_iterations=1,
    reshuffle_each_iteration=True,
    metric='f1',
):
    
    def run_model_again_helper(df_in, shuffled=False):
        all_metric = []
        models = []
        imputers = []
        all_ret_df = []
        df = df_in.copy()
        
        config_dict_best = get_config_dict(config_dict, best_params)
        desc = 'Data' if not shuffled else 'Shuffled'
        for n in tqdm(range(n_total_iterations),desc=desc):
            if shuffled:
                df['sociability'] = np.random.permutation(df['sociability'].values)
            if reshuffle_each_iteration:
                df = df.sample(frac=1)
#             try:
            mean_df, raw_results_df, ret_df, errors_log, models_sub, imputers_sub = run_per_param(
                list_of_variables,
                df,
                config_dict=config_dict_best,
                verbose=verbose,
                n_iterations=n_iterations)

            metric_val = mean_df.loc[mean_df['dataset'] == 'test'][metric].values
            all_metric.append(metric_val)
            models.append(models_sub)
            imputers.append(imputers_sub)
            all_ret_df.append(ret_df)
#             except:
#                 continue
        all_metric = np.array(all_metric)
        all_metric = all_metric.flatten()
        
        models = np.array(models)
        models = models.flatten()
        
        
#         print(imputers)
#         models = list(itertools.chain(*models))
#         imputers = list(itertools.chain(*imputers))


        mean_metric_real_data = np.mean(all_metric)
        std_error_metric_real_data = np.std(all_metric, ddof=1) / np.sqrt(np.size(all_metric))

        print(f'{desc} {metric} Score:{mean_metric_real_data}\u00B1{std_error_metric_real_data}')
        return all_metric, models, imputers, all_ret_df


    all_metric, models, imputers, all_ret_df = run_model_again_helper(df,shuffled=False)
#     print(len(models))
    all_metric_df = pd.DataFrame(all_metric)
    all_metric_df['shuffled'] = False
    
    if run_shuffled:
        all_metric_shuffled,models_shuffled, imputers_shuffled, _ = run_model_again_helper(df,shuffled=True)
        all_metric_shuffled_df = pd.DataFrame(all_metric_shuffled)
        all_metric_shuffled_df['shuffled'] = True
        ret_df = pd.concat([all_metric_df,all_metric_shuffled_df])
    else:
        all_metric_shuffled = []
        ret_df = all_metric_df

    
    
    return all_metric, all_metric_shuffled, ret_df, models, imputers, all_ret_df


In [129]:
def eval_imputer_model_pair_helper(X, y, imputer, model):
    X_imp = imputer.transform(X)
    y_pred = model.predict(X_imp)
    y_prob = model.predict_proba(X_imp)
    y_prob = y_prob[:,0]
    ret_table = eval_model(y, y_pred, y_prob)
    return ret_table
    
def eval_imputer_model_pair(all_df, var_names, n_iterations, imputer_list, model_list):
    X = all_df[var_names].values

    samples_to_remove = ~np.all(np.isnan(X),axis=1)
    y = all_df.iloc[samples_to_remove]['sociability'].values
    filenames = all_df.iloc[samples_to_remove].index.values
    subject_id = all_df.iloc[samples_to_remove]['rat_number'].values
    X = all_df.iloc[samples_to_remove][var_names].values
    

    all_res = []
    for n, (imputer, model) in enumerate(zip(imputer_list, model_list)):
        # split data
        for n_iter in range(n_iterations):
            temp_res = eval_imputer_model_pair_helper(X, y, imputer, model)
            temp_res['pair_index'] = n
            temp_res['model'] = model
            temp_res['imputer'] = imputer
            temp_res['iteration'] = n_iter
            
            all_res.append(temp_res)
    return all_res
   

In [74]:
config_dict_test = {
    'method': ['new'],
    'k_fold': [5],
    'imputer': {
        'iterative_imputer': IterativeImputer,
    },
    'imputer_param': {
        'iterative_imputer': {'max_iter':1000},
    },
    'scaler':{
        'standard_scaler': StandardScaler,
    },
   'model':{
    'random_forest':RandomForestClassifier,
#     'logistic_regression': LogisticRegression,
#     'svm': svm.SVC,
#     'naive_basian': GaussianNB,
#     'knn_classifier': KNeighborsClassifier,
   },
    'model_param':{
    'svm':{'probability': True},
    'knn_classifier':{'n_neighbors': 5}
    }
}

config_dict = {
    'method': ['reuse'],
#     'method': ['reuse', 'new'],
#     'k_fold': [3,4,5],
    'k_fold': [4],
    'imputer': {
#         'missforest':MissForest,
        'iterative_imputer': IterativeImputer,
#         'knn_imputer': KNNImputer        
    },
    'imputer_param': {
        'iterative_imputer': {'max_iter':1000},
    },
    'scaler':{
        'standard_scaler': StandardScaler,
#         'no_scaler': lambda x: x,
    },
    'model':{
        'random_forest':RandomForestClassifier,
#         'logistic_regression': LogisticRegression,
#         'svm': svm.SVC,
#         'naive_basian': GaussianNB,
#         'knn_classifier': KNeighborsClassifier,
    },
    'model_param':{
        'svm':{'probability': True},
        'knn_classifier':{'n_neighbors': 5}
    }
    
}


### Create model using the best sig (pvalue<0.05) and the worst sig (pvalue>0.95)

In [63]:
# current_vars_best = [
#     'lfp_EA_30_80Hz_enc_pre',
#      'lfp_AA_30_80Hz_enc_pre',
#      'coherence_CeA_MeD_30_80Hz_post_pre',
#      'coherence_AA_MeD_4_18Hz_enc_pre',
#      'coherence_CeA_MeD_4_18Hz_enc_pre',
#      'coherence_AA_MeD_30_80Hz_enc_pre',
#      'coherence_CeA_STIA_30_80Hz_enc_pre',
#      'lfp_MePV_4_18Hz_post_pre'
#     ]

In [64]:
# current_vars_best = sig_table_all.loc[sig_table_all['p-value']<=0.05]['Variable'].values
# current_vars_best

In [65]:
# current_vars_best = sig_table_all.loc[sig_table_all['p-value']<=0.05]['Variable'].values
# current_vars_worst = sig_table_all.loc[sig_table_all['p-value']>=0.95]['Variable'].values

# best_params = {
#     'k_fold':4,
#     'model': 'logistic_regression'
# }

# # all_df_shuffled = all_df.sample(frac=1)
# with warnings.catch_warnings(record=True):
#     f1_score_best, f1_score_best_shuffled, ret_df_best,models_best, imputers_best = run_model_again(
#         df=all_df,
#         list_of_variables=current_vars_best,
#         best_params = best_params,
#         config_dict = config_dict,
#         n_total_iterations=100,
#         verbose=False,
#         n_iterations=1,
#         run_shuffled=True
#     )
#     ret_df_best['data'] = 'best'

#     f1_score_worst, f1_score_worst_shuffled, ret_df_worst,models_worst, imputers_worst = run_model_again(
#         df=all_df,
#         list_of_variables=current_vars_worst,
#         best_params = best_params,
#         config_dict = config_dict,
#         n_total_iterations=100,
#         verbose=False,
#         n_iterations=1,
#         run_shuffled=True
#     )
#     ret_df_worst['data'] = 'worst'

In [66]:
# df_f1_combined_results = pd.concat([ret_df_worst, ret_df_best])
# sns.catplot(data= df_f1_combined_results, x='data', y=0, hue='shuffled', kind='violin')

In [115]:
# current_vars = [
#     'lfp_EA_30_80Hz_enc_pre',
#      'lfp_AA_30_80Hz_enc_pre',
#      'coherence_CeA_MeD_30_80Hz_post_pre',
#      'coherence_AA_MeD_4_12Hz_enc_pre',
#      'coherence_CeA_MeD_4_12Hz_enc_pre',
#      'coherence_AA_MeD_30_80Hz_enc_pre',
#      'coherence_CeA_STIA_30_80Hz_enc_pre',
#      'coherence_CeA_MeD_30_80Hz_enc_pre'
#     ]

# best_vars = [
#     'lfp_EA_30_80Hz_enc_pre',
#      'lfp_AA_30_80Hz_enc_pre',
#      'coherence_CeA_MeD_30_80Hz_post_pre',
#      'coherence_AA_MeD_30_80Hz_enc_pre',
#      'coherence_CeA_STIA_30_80Hz_enc_pre',
# ]

best_params = {
    'k_fold':4,
    'model': 'random_forest'
}

# all_df_shuffled = all_df.sample(frac=1)
with warnings.catch_warnings(record=True):

    metric_score_test, metric_score_test_shuffled, ret_df_test, models_test, imputers_test, ret_dt_all_test = run_model_again(
        df=all_df,
        list_of_variables=current_vars,
        best_params = best_params,
        config_dict = config_dict,
        n_total_iterations=100,
        verbose=False,
        n_iterations=1,
        run_shuffled=False,
        metric = 'f1'

    )
    ret_df_test['data'] = 'test'

Data: 100%|██████████████████████████████████████████████████████████████████████████| 100/100 [02:48<00:00,  1.69s/it]

Data f1 Score:0.7487016666666666±0.004955238215707775


In [149]:
ind_best = np.argmax(metric_score_test)
best_models = models_test[ind_best]
best_imputers = imputers_test[ind_best]

import pickle

# save the model to disk
model_paramas = {'model': best_models, 'imputer': best_imputers, 'params': current_vars}
model_filename = osp.join(folder_path, 'best_model_top4_20230626.sav')
pickle.dump(model_paramas, open(model_filename, 'wb'))

In [135]:
# ret_test = pd.concat(ret_dt_all_all)
ret_test = pd.concat(ret_dt_all_test)
ret_test = ret_test.loc[ret_test['dataset']=='test']
ret_test

var_names = current_vars

model_list = ret_test['trained_model'].values
imputer_list = ret_test['trained_imputer'].values

ret = eval_imputer_model_pair(
    all_df,
    var_names,
    n_iterations=1,
    imputer_list=imputer_list,
    model_list=model_list)

In [117]:
# ret

In [136]:
# df_list = [pd.DataFrame(a,index=[a['pair_index']]) for a in ret]
df_list = [pd.DataFrame(a) for a in ret]
df_ret_all = pd.concat(df_list)

In [138]:
metric = 'roc_auc'
best_tab = df_ret_all.iloc[df_ret_all[metric].idxmax()]
print(best_tab)
best_model = best_tab.model
best_imputer = best_tab.imputer
# best_imputer.get_feature_names_out()

accuracy                                                    1.0
precision                                                   1.0
recall                                                      1.0
f1                                                          1.0
kappa                                                       1.0
roc_auc                                                     1.0
pair_index                                                    0
model         DecisionTreeClassifier(max_features='sqrt', ra...
imputer                         IterativeImputer(max_iter=1000)
iteration                                                     0
Name: 0, dtype: object


In [142]:
best_tab = ret_test.iloc[1]
# best_tab = df_ret_all.iloc[]
print(best_tab)
best_model = best_tab.trained_model      
best_imputer = best_tab.trained_imputer                      
# best_imputer.get_feature_names_out()

dataset                                                         test
accuracy                                                       0.818
precision                                                        0.8
recall                                                           0.8
f1                                                               0.8
kappa                                                          0.633
method                                                         reuse
imputer                                            iterative_imputer
scaler                                               standard_scaler
model                                                  random_forest
k_fold                                                             4
ind                                                                1
trained_model      (DecisionTreeClassifier(max_features='sqrt', r...
trained_imputer                      IterativeImputer(max_iter=1000)
trained_scalar                    

In [143]:
import pickle

# save the model to disk
model_paramas = {'model': best_model, 'imputer': best_imputer, 'params': current_vars}
# model_filename = osp.join(folder_path, 'best_model_20230626.sav')
# pickle.dump(model_paramas, open(model_filename, 'wb'))

# imputer_filename = osp.join(folder_path, 'best_imputer.sav')
# pickle.dump(best_imputer, open(imputer_filename, 'wb'))

In [ ]:
with warnings.catch_warnings(record=True):
    best_metric, best_sum_table, best_raw_rable, current_vars, best_ret_df, all_errors_log, best_model, best_imputer, best_params = select_parameters(
        config_dict = config_dict,
        sig_table = None,
        verbose = True,
        df = df_spike_data,
        sig_cut_off =0.6,
        n_iterations=100)

    metric_score_spike, metric_score_spike_shuffled, ret_df_spike = run_model_again(
        df=df_spike_data,
        list_of_variables=current_vars,
        best_params = best_params,
        config_dict = config_dict,
        n_total_iterations=100,
        verbose=False,
        n_iterations=1,
        run_shuffled=True,
        reshuffle_each_iteration=True,
    )
    ret_df_spike['data'] = 'spikes'

In [ ]:
# f1_score_spike, f1_score_spike_shuffled, ret_df_spike = run_model_again(
#     df=df_spike_data,
#     list_of_variables=current_vars,
#     best_params = best_params,
#     config_dict = config_dict,
#     n_total_iterations=100,
#     verbose=False,
#     n_iterations=1,
#     run_shuffled=True
# )
# ret_df_spike['data'] = 'spikes'

In [ ]:

with warnings.catch_warnings(record=True):
    best_metric, best_sum_table, best_raw_rable, current_vars, best_ret_df, all_errors_log, best_model, best_imputer, best_params = select_parameters(
        config_dict = config_dict,
        sig_table = None,
        verbose = True,
        df = df_coherence_data,
        sig_cut_off =0.1,
        n_iterations=100)

    metric_score_coherence,metric_score_coherence_shuffled,ret_df_coherence = run_model_again(
        df=df_coherence_data,
        list_of_variables=current_vars,
        best_params = best_params,
        config_dict = config_dict,
        n_total_iterations=100,
        verbose=False,
        n_iterations=1,
        run_shuffled=True,
        reshuffle_each_iteration=True,

    )
ret_df_coherence['data'] = 'coherence'

In [ ]:
# ret_df_coherence['data'] = 'coherence'


In [ ]:
with warnings.catch_warnings(record=True):
    best_metric, best_sum_table, best_raw_rable, current_vars, best_ret_df, all_errors_log, best_model, best_imputer, best_params = select_parameters(
        config_dict = config_dict,
        sig_table = None,
        verbose = True,
        df = df_lfp_data,
        sig_cut_off =0.1,
        n_iterations=100)

    metric_score_lfp,metric_score_lfp_shuffled,ret_df_lfp = run_model_again(
        df=df_lfp_data,
        list_of_variables=current_vars,
        best_params = best_params,
        config_dict = config_dict,
        n_total_iterations=100,
        verbose=False,
        run_shuffled=True,
        n_iterations=1,
        reshuffle_each_iteration=True,
    )
    
ret_df_lfp['data'] = 'lfp'

In [90]:
with warnings.catch_warnings(record=True):
    best_metric, best_sum_table, best_raw_rable,\
    current_vars, best_ret_df, all_errors_log, best_model, best_imputer, best_params = select_parameters(
        config_dict = config_dict,
        sig_table = None,
        verbose = True,
        df = all_df,
        sig_cut_off =0.1,
        n_iterations=100,
        metric=METRIC
    )

    metric_score_all, metric_score_all_shuffled, ret_df_all, models_all, imputers_all, ret_dt_all_all = run_model_again(
        df=all_df,
        list_of_variables=current_vars,
        best_params = best_params,
        config_dict = config_dict,
        n_total_iterations=100,
        verbose=False,
        run_shuffled=True,
        n_iterations=1,
        reshuffle_each_iteration=True,
        metric=METRIC
    )
    
ret_df_all['data'] = 'all'

============ New param: lfp_AA_30_80Hz_enc_pre (1 of 14) ============


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:56<00:00,  1.76it/s]


0
*********New best kappa was found: 0.14651************
Adding: lfp_AA_30_80Hz_enc_pre
Current vars: ['lfp_EA_30_80Hz_enc_pre', 'lfp_AA_30_80Hz_enc_pre']
method:reuse
imputer:iterative_imputer
model:random_forest
k_fold:4
============ New param: coherence_CeA_MeD_30_80Hz_post_pre (2 of 14) ============


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:14<00:00,  1.35it/s]


0.14651
*********New best kappa was found: 0.3351152882205514************
Adding: coherence_CeA_MeD_30_80Hz_post_pre
Current vars: ['lfp_EA_30_80Hz_enc_pre', 'lfp_AA_30_80Hz_enc_pre', 'coherence_CeA_MeD_30_80Hz_post_pre']
method:reuse
imputer:iterative_imputer
model:random_forest
k_fold:4
============ New param: coherence_AA_MeD_4_12Hz_enc_pre (3 of 14) ============


...

0.5653208020050126
Variable coherence_CeA_MeD_30_80Hz_enc_pre did not improve the model
*************=============ALL DONE============*******************


Data: 100%|██████████████████████████████████████████████████████████████████████████| 100/100 [03:00<00:00,  1.81s/it]


Data kappa Score:0.560465±0.008353638156788062


Shuffled: 100%|██████████████████████████████████████████████████████████████████████| 100/100 [03:25<00:00,  2.05s/it]

Shuffled kappa Score:-0.016472500000000004±0.018515659040574013


In [ ]:
# ret_df_lfp

In [ ]:
df_metric_combined_results = pd.concat([ret_df_all, ret_df_spike, ret_df_lfp, ret_df_coherence])
sns.catplot(data= df_metric_combined_results, x='data', y=0, hue='shuffled', kind='violin')

In [ ]:
# with warnings.catch_warnings(record=True):
#     best_f1, best_sum_table, best_raw_rable, current_vars, best_ret_df, all_errors_log, best_model, best_imputer, best_params = select_parameters(
#         config_dict = config_dict,
#         sig_table = sig_table_all,
#         all_df = all_df, n_iterations=10)

In [ ]:
# best_ret_df.loc[
#     (best_ret_df['imputer'] == 'iterative_imputer')&
#     (best_ret_df['model'] == 'random_forest')&
#     (best_ret_df['kfold'] == 5)&
#     (best_ret_df['dataset'] == 'test')]

# best_raw_rable.loc[
#        (best_raw_rable['imputer'] == 'iterative_imputer')&
#         (best_raw_rable['model'] == 'random_forest')&
#         (best_raw_rable['kfold'] == 5)]
# best_raw_rable.to_clipboard()

In [ ]:
# best_raw_rable.to_clipboard()

In [ ]:
# best_sum_table.to_clipboard()

### Best params selected:



In [ ]:
current_vars

In [ ]:
best_params

In [ ]:
config_dict_best = config_dict.copy()

In [ ]:
# for pname, param in best_params.items():
#     if pname == 'method':
#         continue
#     if pname == 'k_fold':
#         config_dict_best[pname] = [param]
# #         config_dict_best[pname] = [3]
#     else:
#         config_dict_best[pname] = {param:config_dict[pname][param]}

In [ ]:
config_dict_best

### Recreate sucsess

In [ ]:
# config_dict_best = {'method': ['reuse'],
#  'k_fold': [4],
#  'imputer': {'imputer': sklearn.impute._iterative.IterativeImputer},
#  'imputer_param': {'iterative_imputer': {'max_iter': 10000}},
#  'scaler': {'scaler': sklearn.preprocessing._data.StandardScaler},
#  'model': {'model': LogisticRegression},
#  'model_param': {'svm': {'probability': True},
#   'knn_classifier': {'n_neighbors': 5}}}

In [ ]:
# mean_df, raw_results_df, ret_df, errors_log, models, imputers = run_per_param(
#     current_vars, all_df, config_dict=config_dict_best,n_iterations=1)

In [ ]:
# mean_df

In [ ]:
# raw_results_df.loc[raw_results_df['dataset'] == 'test']

In [ ]:
all_df_shuffled = all_df.copy()
all_df_shuffled['sociability'] = np.random.permutation(all_df_shuffled['sociability'].values)

In [ ]:
n_total_iterations = 100
n_iterations = 1

all_metric = []

for n in range(n_total_iterations):
    try:
        mean_df, raw_results_df, ret_df, errors_log, models, imputers = run_per_param(
            current_vars, all_df, config_dict=config_dict_best, n_iterations=n_iterations)
        metric_val = mean_df.loc[mean_df['dataset'] == 'test'][metric].values
        all_metric.append(metric_val)
    except:
        continue
        

In [ ]:
# all_f1

In [ ]:
n_total_iterations = 100
n_iterations = 1

shuffled_all_metric = []

for n in range(n_total_iterations):
    all_df_shuffled = all_df.copy()
    try:
        all_df_shuffled['sociability'] = np.random.permutation(all_df_shuffled['sociability'].values)
        shuffled_mean_df, shuffled_raw_results_df, shuffled_ret_df, shuffled_errors_log, shuffled_models, shuffled_imputers = run_per_param(
            current_vars, all_df_shuffled, config_dict=config_dict_best, n_iterations=n_iterations, verbose=False)
        metric_val = shuffled_mean_df.loc[mean_df['dataset'] == 'test'][metric].values
        shuffled_all_metric.append(metric_val)
    except:
        continue

In [ ]:
shuffled_all_metric = np.array(shuffled_all_metric)
shuffled_all_metric = shuffled_all_metric.flatten()

mean_metric_shuffled_data = np.mean(shuffled_all_metric)
std_error_shuffled_data = np.std(shuffled_all_metric, ddof=1) / np.sqrt(np.size(shuffled_all_metric))

print(f'{metric} Score:{mean_metric_shuffled_data}+-{std_error_shuffled_data}')

In [ ]:
df_metric = pd.DataFrame(zip(all_metric, shuffled_all_metric), columns=['Real Data','Randomized Data'])

In [ ]:
g = sns.violinplot(data=df_metric)
g.set_ylabel(f'Mean {metric} Score')

In [ ]:
from scipy.stats import ttest_rel
real_scores, random_scores = all_metric, shuffled_all_metric

# Calculate the p-value using a paired t-test
_, p_value = ttest_rel(real_scores, random_scores)

# Check if the p-value is below the significance level (e.g., 0.05)
significance_level = 0.05
if p_value < significance_level:
    print("The model's performance on real data is significantly better than random.")
else:
    print("There is no significant difference between the model's performance on real data and random.")

# Quantify the improvement using effect size measures (e.g., Cohen's d)
mean_real = np.mean(real_scores)
mean_random = np.mean(random_scores)
std_real = np.std(real_scores, ddof=1)
effect_size = (mean_real - mean_random) / std_real

print("Effect size (Cohen's d):", effect_size)

In [ ]:
df = best_raw_rable[['GT','predicted','correct','affiliative_level','filenames','dataset']].copy()

df['rat_number'] = df['filenames'].apply(extract_ratnum_from_file_name)
df['day_number'] = df['filenames'].apply(extract_daynum_from_file_name)

In [ ]:
# Percentage of time each rat was in the test or train datasets
rat_dataset_percentage = df.groupby('rat_number')['dataset'].value_counts(normalize=True) * 100
print("Percentage of time each rat was in the test or train datasets:")
print(rat_dataset_percentage)

# Percentage of time each rat was correctly predicted in the test dataset
test_correct_percentage = df[(df['dataset'] == 'test') & (df['correct'])].groupby('rat_number')['correct'].value_counts(normalize=True) * 100
print("\nPercentage of time each rat was correctly predicted in the test dataset:")
print(test_correct_percentage)

# Percentage of time each session was in the test or train datasets
session_dataset_percentage = df.groupby('filenames')['dataset'].value_counts(normalize=True) * 100
print("\nPercentage of time each session was in the test or train datasets:")
print(session_dataset_percentage)

# Percentage of times each session was correctly predicted in the test dataset
test_session_correct_percentage = df[(df['dataset'] == 'test') & (df['correct'])].groupby('filenames')['correct'].value_counts(normalize=True) * 100
print("\nPercentage of times each session was correctly predicted in the test dataset:")
print(test_session_correct_percentage)

In [ ]:
df

In [ ]:
import pandas as pd

# Assuming the dataframe is already loaded and named 'df'

total_count = len(df)  # Total count of samples in the dataset

# Percentage of time each session was in the test or train datasets
session_dataset_percentage = (df.groupby('filenames')['dataset'].value_counts() / total_count) * 100
print("\nPercentage of time each session was in the test or train datasets:")
print(session_dataset_percentage)

# Percentage of times each session was correctly predicted in the test dataset
test_session_correct_percentage = (df[(df['dataset'] == 'test') & (df['correct'])].groupby('filenames')['correct'].value_counts() / total_count) * 100
print("\nPercentage of times each session was correctly predicted in the test dataset:")
print(test_session_correct_percentage)


## Feature selection 

Feature selection steps:

1. Find top 2 candidates
2. Train model and get results
3. Add next candiate and repeat step 2.
4. If the resutls improved, add this featrue, if not, advance to the next candidate

Model trainig steps:

1. Split data

3. Impute missing values
    a. MissForest
    b. IterativeImputer
    
4. rescale data (or not)

5. Train model/s
    a. svm
    b. logistic regression
    c. random forest classifier
    d. knn classifier
    
6. Evaluate model/s


In [ ]:
# imputer_class = MissForest
classifier_class = RandomForestClassifier

pipe = Pipeline([('imputer', imputer_class()), ('scaler', StandardScaler()),('classifier', classifier_class())])

In [ ]:
df_results_test

In [ ]:
df_results_test['dataset'] = 'test'
df_results_train['dataset'] = 'train'
df_results_all = pd.concat([df_results_train,df_results_test])
df_results_all['rat_num'] = df_results_all['files'].apply(extract_ratnum_from_file_name)
df_results_all['day_num'] = df_results_all['files'].apply(extract_day_from_file_name)

In [ ]:
sns.lmplot(data=df_results_all, x='day_num', y='affiliative_level', hue='GT', col='dataset')
sns.lmplot(data=df_results_all, x='day_num', y='affiliative_level', hue='predicted', col='dataset')


## Dimensionality reduction using t-SNE and visualization|

In [ ]:
# # create the figure and axes
# fig, ax = plt.subplots(figsize=(6, 6))

# # add the plots for each dataframe
# df_rats = df_results_all.groupby('rat_num')
# for ratnum , df_r in df_rats:
#     sns.regplot(x='day_num', y='affiliative_level', data=df_r, fit_reg=True, ci=None, ax=ax, label=ratnum)
# ax.set(ylabel='affiliative_level', xlabel='rat_num')
# ax.legend()
# plt.show()

In [ ]:
col_names = df_train.drop(['sociability','dataset'], axis = 1).columns

### Test data

In [ ]:
def plot_pairplot(title, X,y,y_pred):
    df_X = pd.DataFrame(X)


    df_X = df_X.rename(columns={num:cname for num, cname in enumerate(col_names)})
    df_X['GT'] = y
    df_X['predicted'] = y_pred
    df_X['correct'] = df_X['predicted'] == df_X['GT']

    g = sns.pairplot(data=df_X.drop(['predicted','correct'], axis=1), hue='GT')
    g.fig.suptitle(f"{title} Data GT sociability", y=1.05)

    g = sns.pairplot(data=df_X.drop(['GT','correct'], axis=1), hue='predicted')
    g.fig.suptitle(f"{title} Data GT predicted", y=1.05)
    

In [ ]:

X = X_test
y = y_test.values
y_pred = y_test_pred
title = 'Test - Unimputed'
plot_pairplot(title, X,y,y_pred)


X = X_train
y = y_train.values
y_pred = y_train_pred
title = 'Train - Unimputed'
plot_pairplot(title, X,y,y_pred)

X = X_test_imp
y = y_test.values
y_pred = y_test_pred
title = 'Test - Imputed'
plot_pairplot(title, X,y,y_pred)

X = X_train_imp
y = y_train.values
y_pred = y_train_pred
title = 'Train - Imputed'
plot_pairplot(title, X,y,y_pred)


In [ ]:
df_results_test

In [ ]:
df_results_test.columns

In [ ]:
X = X_test_imp

X_embedded = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(X)

df_results_test[['tsne1','tsne2']] = X_embedded
sns.pairplot(df_results_test[['GT','tsne1','tsne2']], hue='GT')
sns.pairplot(df_results_test[['predicted','tsne1','tsne2']], hue='predicted')


X = X_train_imp

X_embedded = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(X)

df_results_train[['tsne1','tsne2']] = X_embedded
sns.pairplot(df_results_train[['GT','tsne1','tsne2']], hue='GT')
sns.pairplot(df_results_train[['predicted','tsne1','tsne2']], hue='predicted')

In [ ]:


X = X_test_imp

X_embedded = TSNE(n_components=1, learning_rate='auto',
                  init='random', perplexity=2).fit_transform(X)

# df_results_test[['tsne1','tsne2']] = X_embedded
# sns.pairplot(df_results_test[['GT','tsne1','tsne2']], hue='GT')
# sns.pairplot(df_results_test[['predicted','tsne1','tsne2']], hue='predicted')

df_results_test[['tsne1']] = X_embedded
df_results_test['affiliative_level_logit'] = df_results_test['affiliative_level'].apply(logit)
sns.lmplot(data=df_results_test, x = 'affiliative_level_logit' ,y = 'tsne1', hue='GT')





# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
X = X_train_imp

X_embedded = TSNE(n_components=1, learning_rate='auto',
                  init='random', perplexity=1).fit_transform(X)

# df_results_test[['tsne1','tsne2']] = X_embedded
# sns.pairplot(df_results_test[['GT','tsne1','tsne2']], hue='GT')
# sns.pairplot(df_results_test[['predicted','tsne1','tsne2']], hue='predicted')

df_results_train[['tsne1']] = X_embedded
df_results_train['affiliative_level_logit'] = df_results_train['affiliative_level'].apply(logit)
sns.(data=df_results_train, x = 'affiliative_level_logit' ,y = 'tsne1', hue='GT')




# Display the plots
plt.tight_layout()
plt.show()

In [ ]:
X = X_train_imp
X_embedded = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(X)

df_results_train[['tsne1','tsne2']] = X_embedded

sns.pairplot(df_results_train[['GT','tsne1','tsne2']], hue='GT')

sns.pairplot(df_results_train[['predicted','tsne1','tsne2']], hue='predicted')

# Display the plots
plt.tight_layout()
plt.show()

In [ ]:
# All data (train and test)


X = np.append(X_train_imp, X_test_imp, axis=0)
y = np.append(y_train, y_test)
y_pred = np.append(y_train_pred, y_test_pred)

X_embedded = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=10, n_iter=5000).fit_transform(X)


df_res_tsne = pd.DataFrame(y, columns=['GT'])
df_res_tsne['dataset'] = np.append(np.full(y_train.shape, "train"), np.full(y_test.shape, "test"))
df_res_tsne['predicted'] = y_pred
df_res_tsne['correct'] = df_res_tsne['predicted'] == df_res_tsne['GT']
df_res_tsne[['tsne1','tsne2']] = X_embedded
#Seaborn pair plot

sns.pairplot(df_res_tsne[['GT','tsne1','tsne2']], hue='GT')

sns.pairplot(df_res_tsne[['predicted','tsne1','tsne2']], hue='predicted')

# # Display the plots
# plt.tight_layout()
# plt.show()

In [ ]:
df_res_tsne.head(1)

### Impute the missing values of all the dataset before training and testing

Performing imputation before splitting the dataset can potentially lead to data leakage and overly optimistic evaluation results. It's generally recommended to split the dataset into training and testing sets before applying any data preprocessing steps, including imputation.

Data leakage can occur when information from the testing set is inadvertently used during the imputation process. This can lead to overfitting and unrealistic evaluation results because the imputation is informed by the target variable in the testing set.

In [ ]:
# imputer = IterativeImputer(max_iter=100)
imputer_class = MissForest

# model_class = RandomForestClassifier
# model = LogisticRegression(max_iter=10000)
# model_class = svm.SVC
model_class = tree.DecisionTreeClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X[:,:3], y, test_size=0.33)

In [ ]:
# Impute data before splitting it into train and test

imputer = imputer_class()
model = model_class()

imputed_data = imputer.fit_transform(X_train)

# # Splitting the imputed data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(imputed_data,
#                                                     labels,
#                                                     test_size=0.3,
#                                                     stratify=labels)


# Creating and training the model (using Logistic Regression as an example)


model.fit(imputed_data, y_train)


# Making predictions on the training and testing sets
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)


# Calculating accuracy scores
# train_accuracy = accuracy_score(y_train.values, y_train_pred)
# test_accuracy = accuracy_score(y_test.values, y_test_pred)
ret_test = eval_model(y_test, y_test_pred)
ret_train = eval_model(y_train, y_train_pred)



df_results_test = pd.DataFrame(y_test, columns={'GT'})
df_results_test['predicted'] = y_test_pred
df_results_test['correct'] = df_results_test['GT'] == df_results_test['predicted']
print(df_results_test)


print('Test')
[print(f'{metric}:{values}') for metric, values in ret_test.items()]
print('Train')
[print(f'{metric}:{values}') for metric, values in ret_train.items()]


### Imputing the data after each splitting

The imputaion quality drops when performing on smaller sub sets

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import accuracy_score



# Splitting the imputed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    labels,
                                                    test_size=0.3,
                                                    stratify=labels)

imputer = imputer_class()
model = model_class()

imputed_data_train = imputer.fit_transform(X_train)
imputed_data_test = imputer.fit_transform(X_test)

# Converting the imputed data back to a DataFrame
imputed_data_train = pd.DataFrame(imputed_data_train, columns=X_train.columns)
imputed_data_test = pd.DataFrame(imputed_data_test, columns=X_test.columns)



# Creating and training the model (using Logistic Regression as an example)
# model = LogisticRegression()
model.fit(imputed_data_train, y_train)

# Making predictions on the training and testing sets
y_train_pred = model.predict(imputed_data_train)
y_test_pred = model.predict(imputed_data_test)

# Calculating accuracy scores
# train_accuracy = accuracy_score(y_train.values, y_train_pred)
# test_accuracy = accuracy_score(y_test.values, y_test_pred)
ret_test = eval_model(y_test, y_test_pred)
ret_train = eval_model(y_train, y_train_pred)


df_results_test = pd.DataFrame(y_test, columns={'GT'})
df_results_test['predicted'] = y_test_pred
df_results_test['correct'] = df_results_test['GT'] == df_results_test['predicted']
print(df_results_test)




print('Test')
[print(f'{metric}:{values}') for metric, values in ret_test.items()]
print('Train')
[print(f'{metric}:{values}') for metric, values in ret_train.items()]



### Cross validation
Since the data plitting have significant effect on the model performance, cross validation is important

In [ ]:
def one_fold(X_train, X_test, y_train, y_test):
        # Splitting the imputed data into training and testing sets


    imputer = imputer_class(max_iter=500)
    model = model_class()
    
    imputed_data_train = imputer.fit_transform(X_train)
    imputed_data_test = imputer.fit_transform(X_test)

    # Converting the imputed data back to a DataFrame
#     imputed_data_train = pd.DataFrame(imputed_data_train, columns=X_train.columns)
#     imputed_data_test = pd.DataFrame(imputed_data_test, columns=X_test.columns)
    
    # debug
#     print(imputed_data_test.shape)
#     print(imputed_data_test.shape)


    # Creating and training the model (using Logistic Regression as an example)
    
    model.fit(imputed_data_train, y_train)

    # Making predictions on the training and testing sets
    y_train_pred = model.predict(imputed_data_train)
    y_test_pred = model.predict(imputed_data_test)
    
    return y_train, y_train_pred, y_test, y_test_pred

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold

# Define the cross-validation strategy
cv = KFold(n_splits=3, shuffle=True)
# cv = StratifiedKFold(n_splits=2, shuffle=True)

cv_scores_train = []
cv_scores_test = []
# for train_index, test_index in cv.split(data, rat_numbers):
for train_index, test_index in cv.split(data):
    X_train, X_test = data.iloc[train_index], data.iloc[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    y_train, y_train_pred, y_test, y_test_pred = one_fold(X_train, X_test, y_train, y_test)
    scores_train = eval_model(y_train, y_train_pred)
    scores_test = eval_model(y_test, y_test_pred)
    cv_scores_train.append(scores_train)
    cv_scores_test.append(scores_test)
# Summarize
ret_train = pd.DataFrame(cv_scores_train).mean()
ret_test = pd.DataFrame(cv_scores_test).mean()






In [ ]:
print('Test')
print(ret_test)

print('Train')
print(ret_train)

# print('Test')
# [[print(f'{metric}:{values}') for metric, values in ret.items()] for ret in cv_scores_test]
# print('Train')
# [[print(f'{metric}:{values}') for metric, values in ret.items()] for ret in cv_scores_train]


In [ ]:
 pd.DataFrame(cv_scores_test)

### Impute the data and see how it looks

In [ ]:
imputer = imputer_class(1000)
imputed_data = imputer.fit_transform(data)

In [ ]:
labels_num = np.array(labels=='affiliative', dtype=float)*0.1
labels_num = labels_num[:, np.newaxis]

In [ ]:

data_conc = np.concatenate((data.values, labels_num), axis=1)
plt.imshow(data_conc, cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
imputed_data_concat = np.concatenate((imputed_data, labels_num), axis=1)
plt.imshow(imputed_data_concat, cmap='hot', interpolation='nearest')
plt.show()


In [ ]:
from sklearn.manifold import TSNE
X = imputed_data
X_embedded = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=15).fit_transform(X)
X_embedded.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder

strings = filenames
label_encoder = LabelEncoder()
converted_numbers = label_encoder.fit_transform(strings)

strings = filenames
label_encoder = LabelEncoder()
label_encoded = label_encoder.fit_transform(labels)

converted_numbers


rat_numbers = [extract_ratnum_from_file_name(filename) for filename in filenames]


In [ ]:
df_tsne = pd.DataFrame(X_embedded)

df_tsne['labels'] = labels
df_tsne['filenames'] = converted_numbers
df_tsne['rat_number'] = rat_numbers
# df_tsne['pred'] = 


In [ ]:
# get the dummies and store it in a variable
dummies = pd.get_dummies(df_tsne.labels)
 
# Concatenate the dummies to original dataframe
merged = pd.concat([df_tsne, dummies], axis='columns')
 
# drop the values
merged.drop(['labels'], axis='columns')

merged = merged[['rat_number','affiliative','aversive']].groupby('rat_number').sum()
merged['sum'] = merged['affiliative'] + merged['aversive']
merged

In [ ]:
sns.scatterplot(data=df_tsne, x=0, y=1, hue='labels')

In [ ]:
# sns.catplot(data=df_tsne, x=0, y=1, hue='rat_number', row='labels')

In [ ]:
# sns.catplot(data=df_tsne, x=0, y=1, hue='filenames', row='labels')

In [ ]:
G = nx.Graph()
all_pos = {}
colors = []
for r, df_r in df_tsne.groupby('rat_number'):


    f_names = df_r['filenames'].values
    pos_ = df_r[[0,1]].values
    pos_2 = {(r,f): p for p, f in zip(pos_,f_names)}
    mean_pos = np.mean(pos_,axis=0)
    pos_2[r] = mean_pos
    
    labels_sub = np.array(['b']* len(df_r))
    labels_sub[np.where(np.array(df_r['labels']=='affiliative'))] = 'g'
    
    all_pos.update(pos_2)
    [G.add_edge(r, (r,f)) for f in f_names]
    colors = np.append(colors, labels_sub)
# colors = np.array(colors).flatten()
fig = plt.figure(figsize=(40,80))
# nx.draw_networkx(G,all_pos)
nx.draw_networkx(G,all_pos, edge_color=colors, font_size=50,width=5)
ax = plt.gca()
ax.margins(0.08)
plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# G = nx.Graph()
# all_pos = {}
# df_g = df_tsne.groupby('rat_number')
# n_groups = len(df_g)
# fig = plt.figure(figsize=(40,80))
# for n, (r, df_r) in enumerate(df_g):
#     G = nx.Graph()
#     plt.subplot(n_groups,1,n+1)

#     f_names = df_r['filenames'].values
#     pos_ = df_r[[0,1]].values
#     labels_sub = np.array(['b']* len(df_r))
#     labels_sub[np.where(np.array(df_r['labels']=='affiliative'))] = 'g'
# #     labels_sub = np.append(labels_sub, 'y', axis=None)
    
    
#     pos_2 = {(r,f): p for p, f in zip(pos_,f_names)}
#     mean_pos = np.mean(pos_,axis=0)
    
#     pos_2[r] = mean_pos
    
#     all_pos.update(pos_2)
#     [G.add_edge(r, (r,f)) for f in f_names]
    

#     nx.draw_networkx(G,pos_2, node_size=2000, font_size=50, edge_color=labels_sub)
#     ax = plt.gca()
#     ax.margins(0.08)
# #     plt.axis("off")
#     plt.tight_layout()
# plt.show()